In [13]:
import h5py
import numpy as np


path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/cA2.09.48.h5'
with h5py.File(path,'w') as fw:
    path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
    with h5py.File(path) as f:
        for key in f.keys():
            fw.copy(f[key],fw,name=key)
            print(fw[key])
    
    path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_tensor.h5'
    with h5py.File(path) as f:
        t=f['VEV']['pi0f']['data']['pi0'][:]
        fw.copy(f['VEV']['pi0f']['data']['pi0'],fw,name='data/vev')

<HDF5 dataset "cfgs": shape (1228,), type "|O">
<HDF5 group "/data" (3 members)>
<HDF5 dataset "moms": shape (2, 3), type "<i8">


In [111]:
cfg2new=lambda cfg:{'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]]+cfg[:4]
symExpand=lambda dat:np.concatenate([dat,np.flip(dat,axis=1)[:,1:-1]],axis=1)

for ens in ['B64','C80','D96']:
    path=f'/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/{ens}.h5'
    with h5py.File(path,'w') as fw:
        path=f'/p/project/pines/li47/code/temp/{ens}.txt'
        with open(path,'r') as f:
            t=f.read()
            n=54
            data={}
            cfgs=[]; times=[]
            for i in range(len(t)//n):
                tt=t[i*n:(i+1)*n-1]
                tt=tt.split(' ')
                cfg=cfg2new(tt[0]); cfgs.append(cfg)
                time=int(tt[-4]); times.append(time)
                data[(cfg,time)]=tt[-3:]
        cfgs=list(set(cfgs)); times=list(set(times))
        cfgs.sort(); times.sort()
        print(len(cfgs),len(times))
        fw.create_dataset('cfgs',data=cfgs)
        fw.create_dataset('moms',data=[[0,0,0]])
        
        t=np.array([[np.float64(data[(cfg,time)][0]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/pi0_disc',data=t)
        
        t=np.array([[np.float64(data[(cfg,time)][1]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/pi0_conn',data=t)
        
        t=np.array([[np.float64(data[(cfg,time)][2]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/piC',data=t)
        
        fw.create_dataset('data/vev',data=np.zeros(len(cfgs)))
            

751 65
400 81
373 97
